In [3]:
import requests
import json
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import smtplib
from email.mime.text import MIMEText
def fetch_price_data():
    with open('data.json', encoding='utf8') as f:
        price_data = f.read()
    if price_data is not None:
        json_price_data = json.loads(price_data)
        return json_price_data
    else:
        return None

def convert_currency(amount, from_currency, to_currency):
    url = f"https://api.apilayer.com/fixer/convert?to={to_currency}&from={from_currency}&amount={amount}"
    headers = {"apikey": "iXnZA0HZ2IS1PsZlgJuun35oPHaGhdoq"}
    response = requests.request("GET", url, headers=headers)
    if response.status_code == 200:
        result = json.loads(response.text)
        return result['result']
    else:
        return None

def suggest_store(product_name):
    with open('data.json', encoding='utf8') as f:
        price_data = f.read()
    if price_data is not None:
        json_price_data = json.loads(price_data)
        sid = SentimentIntensityAnalyzer()
        scores = {}
        for product in json_price_data:
            if product_name.lower() in product['name'].lower():
                reviews = product.get('reviews', [])
                scores[product['name']] = 0
                for review in reviews:
                    sentiment_scores = sid.polarity_scores(review['text'])
                    if sentiment_scores['compound'] >= 0.05:
                        scores[product['name']] += 1
                    elif sentiment_scores['compound'] <= -0.05:
                        scores[product['name']] -= 1
                scores[product['name']] /= len(reviews) or 1
        if scores:
            best_store = max(scores, key=scores.get)
            print(f"\nBased on the sentiment analysis of the reviews, the best store to buy {product_name} from is {best_store}.")
        else:
            print("No matching products found. Please enter a valid product name.")
    else:
        print("Error reading price data.")

def send_email(to_address, subject, body):
    # Email account details
    from_address = 'comparisonprice40@gmail.com'
    password = 'tpcmolpatrwjbpxf'
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    # Create the email message
    message = MIMEText(body)
    message['From'] = from_address
    message['To'] = to_address
    message['Subject'] = subject

    # Log in to the email account and send the message
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(from_address, password)
        server.sendmail(from_address, to_address, message.as_string())

def update_price_data(updated_price_data):
    with open('data.json', 'w', encoding='utf8') as f:
        json.dump(updated_price_data, f, ensure_ascii=False, indent=4)

        
if __name__ == '__main__':
    json_price_data = fetch_price_data()

    # Get the product name from the user input
    print("Products available:")
    for product in json_price_data:
        print(product['name'])
    partial_input = True
    while partial_input:
        product_name = input("Enter the name of the product you want to compare prices for: ")
        matched_products = []
        for product in json_price_data:
            if product_name.lower() in product['name'].lower():
                matched_products.append(product)
        if matched_products:
            partial_input = False
        else:
            print("No matching products found. Please enter a valid product name.")

    # Loop through the matched products and compare prices
    for product in matched_products:
        prices = []
        prices.append({'name': product['name'], 'price': product['amazon_price'], 'url': product['amazon_url']})
        prices.append({'name': product['name'], 'price': product['walmart_price'], 'url': product['walmart_url']})
        prices.append({'name': product['name'], 'price': product['ebay_price'], 'url': product['ebay_url']})
        sorted_prices = sorted(prices, key=lambda x: float(x['price']))
        print(f"\nPrices for {product['name']}:\n")
        for price in sorted_prices:
            store_name = ''
        # Pick the store name based on url
            if 'amazon' in price['url'].lower():
                store_name = 'Amazon'
            elif 'walmart' in price['url'].lower():
                store_name = 'Walmart'
            elif 'ebay' in price['url'].lower():
                store_name = 'eBay'
            print(f"${price['price']} at {store_name} {price['url']}\n")
            
           
        minPricedItem = sorted_prices[0]
        store_name = ''
        # Pick the store name based on url
        if 'amazon' in minPricedItem['url'].lower():
            store_name = 'Amazon'
        elif 'walmart' in minPricedItem['url'].lower():
            store_name = 'Walmart'
        elif 'ebay' in minPricedItem['url'].lower():
            store_name = 'eBay'
        # Convert to a different currency
        currency = input("\nEnter the currency you want to convert to (e.g. INR): ")
        converted_price = convert_currency(minPricedItem['price'], 'USD', currency)
        if converted_price is not None:
            print(f"\n{minPricedItem['name']} is available at the cheapest price of {converted_price} {currency} at {store_name}.")
        else:
            print("Error converting currency.")
        
        # Check if the price has changed
        if 'current_price' in product and minPricedItem['price'] != product['current_price']:
            subject = f"{product['name']} price has changed!"
            body = f"{product['name']} is now available for ${minPricedItem['price']} at {store_name}."
            # Update the price data with the new price
            product['current_price'] = minPricedItem['price']
            update_price_data(json_price_data)
            # Send an email notification about the price change
            to_address = '2006254@kiit.ac.in'  # Replace with your own email address
            send_email(to_address, subject, body)
            print(f"\nPrice has changed for {product['name']}! Email notification sent.")
        elif 'current_price' not in product:
            # if current_price is not in product, this is the first time we are fetching prices, we don't send an email
            product['current_price'] = minPricedItem['price']
            update_price_data(json_price_data)
        else:
            print(f"\nLowest price for {product['name']} has not changed. No email notification sent.")

        
# Suggest a product store based on reviews
        good_keywords = ['good', 'great', 'awesome', 'excellent']
        bad_keywords = ['bad', 'poor', 'terrible', 'horrible']
        reviews = []
        reviews.append({'name': 'Amazon', 'review': product['amazon_review']})
        reviews.append({'name': 'Walmart', 'review': product['walmart_review']})
        reviews.append({'name': 'eBay', 'review': product['ebay_review']})
        positive_reviews = []
        negative_reviews = []
        for review in reviews:
            if review['review'] is not None:
                if any(keyword in review['review'].lower() for keyword in good_keywords):
                    positive_reviews.append(review)
                if any(keyword in review['review'].lower() for keyword in bad_keywords):
                    negative_reviews.append(review)
        if positive_reviews and not negative_reviews:
            suggested_store = positive_reviews[0]['name']
            print(f"\nBased on reviews, {suggested_store} is suggested as the best store to buy {product['name']}.")
        elif not positive_reviews and negative_reviews:
            suggested_store = negative_reviews[0]['name']
            print(f"\nBased on reviews, {suggested_store} is suggested as the worst store to buy {product['name']}.")
        elif positive_reviews and negative_reviews:
            pos_review_scores = []
            neg_review_scores = []
            for review in positive_reviews:
                pos_review_scores.append((review['name'], review['review'].count('good') + review['review'].count('great')))
            for review in negative_reviews:
                neg_review_scores.append((review['name'], review['review'].count('bad') + review['review'].count('poor')))
            pos_review_scores = sorted(pos_review_scores, key=lambda x: x[1], reverse=True)
            neg_review_scores = sorted(neg_review_scores, key=lambda x: x[1], reverse=True)
            if pos_review_scores[0][1] > neg_review_scores[0][1]:
                suggested_store = pos_review_scores[0][0]
                print(f"\nBased on reviews, {suggested_store} is suggested as the best store to buy {product['name']}.")
            elif pos_review_scores[0][1] < neg_review_scores[0][1]:
                suggested_store = neg_review_scores[0][0]
                print(f"\nBased on reviews, {suggested_store} is suggested as the worst store to buy {product['name']}.")
            else:
                print("\nCould not determine a suggested store based on reviews.")

Products available:
Samsung Galaxy S23
Apple iPhone 12
Google Pixel 6a
Samsung Galaxy Watch 5
Canon EOS 2000D (Rebel T7) DSLR Camera
Sony WH-1000XM4 Wireless Headphones
Air Jordan 1 Low 'Bred Toe' Black/Red
Le Creuset Enameled Cast Iron Dutch Oven
Sony Playstation 5 PS5 
Amazon Echo Dot
Fitbit fitness tracker
The Alchemist
Bose QuietComfort headphones
'To Kill a Mockingbird' by Harper Lee
Casio FX-300 Scientific Calculator
Harry Potter and the Sorcerer's Stone
Bose Noise Cancelling Headphones 700
Sony SRSXB23 EXTRA BASS™ Portable BLUETOOTH® Speaker 
Intel Core i9-10900K Desktop Processor 10 Cores up to 5.3 GHz Unlocked 
OnePlus 11 5G
Bean Bag 
Apple 10.2-inch iPad
Apple iPad Air 12.
Apple Pencil(1st Generation)
Apple Airtag(Pack of 4)
Wings of Fire: An Autobiography
Chanakya Neeti
Meditations: A New Translation
ONEPLUS 10T | Moonstone Black |
GoPro HERO9
PlayStation 5 Console
Fitbit Versa 4 Fitness Smartwatch
Sandisk 64 GB pendrive
SanDisk MicroSD Card Ultra SD Memory Card 128GB
Enter 